In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv("/content/drive/MyDrive/Fraud_Detection_data/Credit_Card.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [3]:
data.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [8]:
data.type.value_counts()

PAYMENT     52865
CASH_OUT    44255
CASH_IN     27997
TRANSFER    11712
DEBIT        1286
Name: type, dtype: int64

In [15]:
import plotly.express as px

type = data["type"].value_counts()
transactions = type.index
quantity = type.values

fig = px.pie(data,values=quantity,names=transactions,hole =0.5,
             title="Transactions Type")
fig.show()


In [16]:
correlation = data.corr()
correlation["isFraud"].sort_values(ascending=False)


<ipython-input-16-87e95aef3cd8>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



isFraud           1.000000
amount            0.033093
oldbalanceOrg    -0.003436
newbalanceDest   -0.005799
oldbalanceDest   -0.008594
newbalanceOrig   -0.009190
step             -0.050135
isFlaggedFraud         NaN
Name: isFraud, dtype: float64

In [17]:
data["type"] = data["type"].map({"CASH_OUT":1,"PAYMENT":2,
                                 "CASH_IN":3,"TRANSFER":4,
                                 "DEBIT":5})
data["isFraud"]= data["isFraud"].map({0:"Not a Fraud Transaction",1:"Fraud Transaction"})

In [18]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,2,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,Not a Fraud Transaction,0.0
1,1,2,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,Not a Fraud Transaction,0.0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,Fraud Transaction,0.0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,Fraud Transaction,0.0
4,1,2,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,Not a Fraud Transaction,0.0


In [23]:
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.ensemble import RandomForestClassifier

X = data.drop(["isFraud","nameOrig","nameDest"],axis=1)
y = data["isFraud"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

clf= RandomForestClassifier()
clf.fit(X_train,y_train)

#clf.score(X_test,y_test)
crossval=cross_val_score(clf,X,y,cv=5)

In [24]:
crossval

array([0.51757593, 0.99971039, 0.99942077, 0.99920356, 0.99920356])

In [25]:
print(np.mean(crossval))

0.9030228432827716


In [27]:
features = np.array([[68, 1,5800988, 5800988, 0,0,5800988,0]])
clf.predict(features)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



array(['Fraud Transaction'], dtype=object)